In [1]:
import numpy as np
import importlib
import nbimporter

import vapour_pressure as vp
import MoistureConversion as mc

Importing Jupyter notebook from MoistureConversion.ipynb


### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$\dot q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  
#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$\dot m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$\dot m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$\dot m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$\dot m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$\dot m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$\dot m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


### 取得した物性値を流量計算に適した形に換算するクラス

#### 物性名からインポートすべきモジュール名を検索する

In [2]:
def getModuleName( name ):
    try:
        return {
            'BentheimerSandstone': 'BentheimerSandstone',        
        }[name]
    except:
        raise ValueError('物性リストに登録されていない物性名が指定されました。')    

#### インポートすべきモジュール名をもとに物性値計算を定義した外部ファイルをインポートする

In [3]:
def importModule( moduleName ):
    try:
        return importlib.import_module( moduleName )
    except:
        raise ImportError('物性リストに登録されている外部モジュールが見つかりません。')

#### 水の気体定数

In [4]:
def Rv():
    return 8316.0/18.016

#### 水の密度

In [5]:
def row():
    return 1000.0

#### 重力加速度

In [6]:
def grav():
    return 9.806650

In [7]:
def getDPDM( temp, miu ):
    return vp.Pvs( temp ) * np.exp( miu / Rv() / temp ) / Rv() / temp

In [8]:
def getDPDT( temp, miu ):
    return np.exp( miu / Rv() / temp ) * ( vp.DPvs( temp ) - vp.Pvs( temp ) * miu / Rv() / ( temp**2 ) )

In [9]:
class MaterialKernel():
    # このクラスは Material そのものを表すのではなくて、あくまで CROW 等の物性を計算するアルゴリズムを提供するだけなので、
    # 名前に Kernel とつけました。
    
    def __init__( self, name ):
        
        # 物性名称からインポートすべきモジュール名称を検索する。
        moduleName = getModuleName( name )
        
        # インポートすべきモジュールをインポートする。モジュールが存在しなければエラー。
        module = importModule( moduleName )
        
        if ( 'getCRowByMiu' in dir( module ) ) and ( 'getCRowByRH' in dir( module ) ):
            # module 内に、関数 getCRowByMiu と getCRowByRH が両方定義されているとき
            # クラス Material に両関数を追加する。
            self.getCRowByMiu = module.getCRowByMiu
            self.getCRowByRH  = module.getCRowByRH
        elif ( 'getCRowByMiu' in dir( module ) ):
            # module 内に、関数 getCRowByMiu のみ定義されているとき
            # クラス Material に getCRowByMiu を追加し、getCRowByRH 新たに作成して追加する。
            self.getCRowByMiu = module.getCRowByMiu
            self.getCRowByRH  = convertMiuFuction2RHFunction( module.getCRowByMiu )
        elif ( 'getCRowByRH' in dir( module ) ):
            # module 内に、関数 getCRowByRH のみ定義されているとき
            # クラス Material に getCRowByRH を追加し、getCRowByMiu 新たに作成して追加する。
            self.getCRowByMiu = convertRHFunction2MiuFunction( module.getCRowByRH )
            self.getCRowByRH  = module.getCRowByRH
        else:
            # どちらも存在しない場合は関数を追加しない。
            pass
        # 以下、同じ。
        
        if( 'getLAMByMiu' in dir( module ) ) and ( 'getLAMByRH' in dir( module ) ):
            self.getLAMByMiu = module.getLAMByMiu
            self.getLAMByRH  = module.getLAMByRH
        elif( 'getLAMByMiu' in dir( module ) ):
            self.getLAMByMiu = module.getLAMByMiu
            self.getLAMByRH  = convertMiuFuction2RHFunction( module.getLAMByMiu )
        elif( 'getLAMByRH' in dir( module ) ):
            self.getLAMByMiu = convertRHFunction2MiuFunction( module.getLAMByRH )
            self.getLAMByRH  = module.getLAMByRH
        else:
            pass

        if( 'getPhiByMiu' in dir( module ) ) and ( 'getPhiByRH' in dir( module ) ):
            self.getPhiByMiu = module.getPhiByMiu
            self.getPhiByRH  = module.getPhiByRH
        elif( 'getPhiByMiu' in dir( module ) ):
            self.getPhiByMiu = module.getPhiByMiu
            self.getPhiByRH  = convertMiuFuction2RHFunction( module.getPhiByMiu )
        elif( 'getPhiByRH' in dir( module ) ):
            self.getPhiByMiu = convertRHFunction2MiuFunction( module.getPhiByRH )
            self.getPhiByRH  = module.getPhiByRH
        else:
            pass

        if( 'getDwByMiu' in dir( module ) ) and ( 'getDwByRH' in dir( module ) ):
            self.getDwByMiu = module.getDwByMiu
            self.getDwByRH  = module.getDwByRH
        elif( 'getDwByMiu' in dir( module ) ):
            self.getDwByMiu = module.getDwByMiu
            self.getDwByRH  = convertMiuFuction2RHFunction( module.getDwByMiu )
        elif( 'getDwByRH' in dir( module ) ):
            self.getDwByMiu = convertRHFunction2MiuFunction( module.getDwByRH )
            self.getDwByRH  = module.getDwByRH
        else:
            pass

        if( 'getDPByMiu' in dir( module ) ) and ( 'getDPByRH' in dir( module ) ):
            self.getDPByMiu = module.getDPByMiu
            self.getDPByRH  = module.getDPByRH
        elif( 'getDPByMiu' in dir( module ) ):
            self.getDPByMiu = module.getDPByMiu
            self.getDPByRH  = convertMiuFuction2RHFunction( module.getDPByMiu )
        elif( 'getDPByRH' in dir( module ) ):
            self.getDPByMiu = convertRHFunction2MiuFunction( module.getDPByRH )
            self.getDPByRH  = module.getDPByRH
        else:
            pass

        if( 'getDPhiByMiu' in dir( module ) ) and ( 'getDPhiByRH' in dir( module ) ):
            self.getDPhiByMiu = module.getDPhiByMiu
            self.getDPhiByRH  = module.getDPhiByRH
        elif( 'getDPhiByMiu' in dir( module ) ):
            self.getDPhiByMiu = module.getDPhiByMiu
            self.getDPhiByRH  = convertMiuFuction2RHFunction( module.getDPhiByMiu )
        elif( 'getDPhiByRH' in dir( module ) ):
            self.getDPhiByMiu = convertRHFunction2MiuFunction( module.getDPhiByRH )
            self.getDPhiByRH  = module.getDPhiByRH
        else:
            pass

        if( 'getMiuByPhi' in dir( module ) ) and ( 'getRHByPhi' in dir( module ) ):
            self.getMiuByPhi = module.getMiuByPhi
            self.getRHByPhi  = module.getRHByPhi
        elif( 'getMiuByPhi' in dir( module ) ):
            self.getMiuByPhi = module.getMiuByPhi
            self.getRHByPhi  = lambda temp, phi : mc.convertMiu2RH( temp, module.getMiuByPhi( temp, phi ) )
        elif( 'getRHByPhi' in dir( module ) ):
            self.getMiuByPhi = lambda temp, phi : mc.convertRH2Miu( temp, module.getRHByPhi( temp, phi ) )
            self.getRHByPhi  = module.getRHByPhi
        else:
            pass
        
        if( 'getLDMGByMiu' in dir(module) ) and ( 'getLDMGByRH' in dir( module ) ):
            self.getLDMGByMiu = module.getLDMGByMiu
            self.getLDMGByRH  = module.getLDMGByRH
        elif( 'getLDMGByMiu' in dir( module ) ):
            self.getLDMGByMiu = module.getLDMGByMiu
            self.getLDMGByRH  = convertMiuFuction2RHFunction( module.getLDMGByMiu )
        elif( 'getLDMGByRH' in dir( module ) ):
            self.getLDMGByMiu = convertRHFuction2MiuFunction( module.getLDMGByRH )
            self.getLDMGByRH  = module.getLDMGByRH
        elif( 'getDPByMiu' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDMG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDMGByMiu = lambda temp, miu : getDPDM( temp, miu                    ) * self.getDPByMiu( temp, miu                    )
            self.getLDMGByRH  = lambda temp, rh  : getDPDM( temp, mc.convertRH2Miu( rh ) ) * self.getDPByMiu( temp, mc.convertRH2Miu( rh ) )
        else:
            pass
        
        if( 'getLDTGByMiu' in dir(module) ) and ( 'getLDTGByRH' in dir( module ) ):
            self.getLDTGByMiu = module.getLDTGByMiu
            self.getLDTGByRH  = module.getLDTGByRH
        elif( 'getLDTGByMiu' in dir( module ) ):
            self.getLDTGByMiu = module.getLDTGByMiu
            self.getLDTGByRH  = convertMiuFuction2RHFunction( module.getLDTGByMiu )
        elif( 'getLDTGByRH' in dir( module ) ):
            self.getLDTGByMiu = convertRHFuction2MiuFunction( module.getLDTGByRH )
            self.getLDTGByRH  = module.getLDTGByRH
        elif( 'getDPByMiu' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDTG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDTGByMiu = lambda temp, miu : getDPDT( temp, miu                    ) * self.getDPByMiu( temp, miu                    )
            self.getLDTGByRH  = lambda temp, rh  : getDPDT( temp, mc.convertRH2Miu( rh ) ) * self.getDPByMiu( temp, mc.convertRH2Miu( rh ) )
        else:
            pass

        if( 'getLDMLByMiu' in dir(module) ) and ( 'getLDMLByRH' in dir( module ) ):
            self.getLDMLByMiu = module.getLDMLByMiu
            self.getLDMLByRH  = module.getLDMLByRH
        elif( 'getLDMLByMiu' in dir( module ) ):
            self.getLDMLByMiu = module.getLDMLByMiu
            self.getLDMLByRH  = convertMiuFuction2RHFunction( module.getLDMLByMiu )
        elif( 'getLDMLByRH' in dir( module ) ):
            self.getLDMLByMiu = convertRHFuction2MiuFunction( module.getLDMLByRH )
            self.getLDMLByRH  = module.getLDMLByRH
        elif( 'getDwByMiu' in dir( module ) ):
            # 本クラスに Dw を計算する式が存在すれば、LDML の計算を Dw を用いて行う。
            # getDwByRH が定義されていれば、getDwByMiu も定義されていれはずなので、ここでは getDwByMiu 関数のみ検索する。
            self.getLDMLByMiu = lambda temp, miu : row() / grav() * self.getDwByMiu( temp, miu                    )
            self.getLDMLByRH  = lambda temp, rh  : row() / grav() * self.getDwByMiu( temp, mc.convertRH2Miu( rh ) )
        else:
            pass        

In [10]:
def convertMiuFuction2RHFunction( miuFunc ):
    def rhFunc( temp, rh ):
        miu = mc.convertRH2Miu( temp, rh )
        return miuFunc( temp, miu )
    return rhFunc

In [11]:
def convertRHFunction2MiuFunction( rhFunc ):
    def miuFunc( temp, miu ):
        rh = mc.convertMiu2RH( temp, miu )
        return rhFunc( temp, rh )
    return miuFunc

In [12]:
m = MaterialKernel('BentheimerSandstone')


Importing Jupyter notebook from BentheimerSandstone.ipynb
Importing Jupyter notebook from van_Genuchten.ipynb


In [13]:
m.getCRowByMiu( 293.15, -10.0 )

2106282.314793056

In [14]:
mc.convertMiu2RH( 293.15, -10.0 )

0.99992610110464986

In [15]:
m.getCRowByRH( 293.15, 0.99992610110464986 )

2106282.3147931653